### Explaination of the face detection code with OpenCV

In [1]:
#import necessary libraries
import cv2

# Load the pre-trained Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Draw rectangles around each face
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face Detection', frame)

    # Break the loop with the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()


: 

## Custom face detection with optional webcam or ESP32 integration

In [3]:
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import db
import requests
import face_recognition
import cv2
import os
import numpy as np
import firebase
from firebase import firebase
import time
import urllib.request


# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
known_face_encodings = []
known_face_names = []


# Load sample pictures and learn how to recognize them
def load_known_faces(known_faces_dir='/add your directory here/faces'):
    for name in os.listdir(known_faces_dir):
        person_dir = os.path.join(known_faces_dir, name)
        # Check if it's a directory
        if os.path.isdir(person_dir):
            for filename in os.listdir(person_dir):
                image_path = os.path.join(person_dir, filename)
                # Check if the file is an image
                if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image = face_recognition.load_image_file(image_path)
                    # Sometimes you might get an image without any recognizable faces.
                    # face_encodings function returns a list, and you should check if it's not empty.
                    face_encodings = face_recognition.face_encodings(image)
                    if face_encodings:
                        encoding = face_encodings[0]
                        known_face_encodings.append(encoding)
                        known_face_names.append(name)

# instance of function 

load_known_faces()

# for webcam initialize set it to zero 
# Initialize the webcam
# Initialize the ESP32-cam 

# setting ESP32 camera url server

cam_url = "http://192.168.1.7/cam-hi.jpg"


video_capture = cv2.VideoCapture(cam_url)

video_capture.set(3, 640)
video_capture.set(4, 480)

firebase = firebase.FirebaseApplication('https://ecocleaner-3222b-default-rtdb.firebaseio.com/', None)

# check if the hit is successful
trigger = firebase.get('/ecocleaner-3222b-default-rtdb', None)
if trigger == 1:
    while True:
        # Capture frame-by-frame
        # ret, frame = video_capture.read()
        # # Load an image from the webcam and locate the faces


        img_resp = urllib.request.urlopen(cam_url)
        imgnp = np.array(bytearray(img_resp.read()), dtype=np.uint8)
        im = cv2.imdecode(imgnp, -1)

        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(im)
        face_encodings = face_recognition.face_encodings(im, face_locations)

        face_names = []
        
        for face_encoding in face_encodings:
            # See if the face is a match for the known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

            # the default name value should appear if find not similar face which will be sent to the database as unkown
            name = "unknown" # set defualt to unknown

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                # updating the name from 'unknown' to the best match name
                name = known_face_names[best_match_index]
                
            # time.sleep(30)
            face_names.append(name)

            
            final_results = firebase.post('/ecocleaner-3222b-default-rtdb', face_names)
            print(final_results)




        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Draw a box around the face
            cv2.rectangle(im, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(im, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            # sending the outout to the database just after catching the name 
            
            ## it fails here with this logic 

            # we will try here to send the output to the database

        # Display the resulting image
        cv2.imshow('Video', im)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close windows
    video_capture.release()
    cv2.destroyAllWindows()


{'name': '-NvhZwmoZYw0R3Gli0Bt'}
{'name': '-NvhZx-swF92j29mlpkX'}
{'name': '-NvhZxCizKuRhSTCAVTp'}
{'name': '-NvhZxNJZqI5ulwaB6MH'}
{'name': '-NvhZxZmsEY7wDUraqXS'}
{'name': '-NvhZxj4YsVx4kBqLapg'}
{'name': '-NvhZxtS2k-TtVgVWYpj'}
{'name': '-NvhZy2lUAHhU2kTllA6'}
{'name': '-NvhZyCvDKFIwqKspWDf'}
{'name': '-NvhZyNJhSJp-fcFB1xN'}
{'name': '-NvhZyXYCKvfxVdDseAu'}
{'name': '-NvhZyiKXpo5l2dwWvBe'}
{'name': '-NvhZytjKbxveB3tvYU1'}
{'name': '-NvhZz2zbaUy_Gz9RKKQ'}
{'name': '-NvhZzDHGWLE7IV1hxA9'}
{'name': '-NvhZzNVfd8ncwT9uE8K'}
{'name': '-NvhZzXlz_O1tR9l8_LE'}
{'name': '-NvhZzhCORfrZSrFv9JS'}
{'name': '-NvhZzrdR4dYGDH_A1nq'}
{'name': '-Nvh_-22L29FV73K7Kjq'}
{'name': '-Nvh_-CpRE1aAYcatSgx'}
{'name': '-Nvh_-N-aB5z66SU_fZj'}
{'name': '-Nvh_-YRe-x9Lp_mT04H'}
{'name': '-Nvh_-hVvm0_9o00X4NX'}
{'name': '-Nvh_-tZ6fUJWr4_sXAh'}
{'name': '-Nvh_03hrUWTfx7DKSdm'}
{'name': '-Nvh_0HuB27Qesre4vFk'}
{'name': '-Nvh_0SNtLxYo1yh6eVl'}
{'name': '-Nvh_0ccnWtYgrhpVXkc'}
{'name': '-Nvh_0mwvnJ8iM8ppZnl'}
{'name': '

: 